In [1]:
%run 0_parameters.ipynb


WELCOME



In [2]:
import glob
import bayesianchangepoint as bcp

In [3]:
msfigpath = '../PasturelMontagniniPerrinet2019/figures'

In [4]:
from aSPEM import Analysis
e = Analysis(name_file_fit='fct_velocity_sigmo')

# find_h

In [5]:
import pickle

In [6]:
def r_(p, data) :
    
    p = np.ma.masked_array(p, mask=np.isnan(data)).compressed()
    data = np.ma.masked_array(data, mask=np.isnan(data)).compressed()

    from scipy import stats
    slope, intercept, r_, p_value, std_err = stats.linregress(p, data)
    
    return r_

In [7]:
def mutual_information(p, data, bin_p=5, sub_data=40):
    
    x = np.ma.masked_array(p, mask=np.isnan(data)).compressed()
    y = np.ma.masked_array(data, mask=np.isnan(data)).compressed()

    support_x = np.linspace(min(x), max(x), bin_p)
    support_y = np.linspace(min(y), max(y), int(len(data)/sub_data))

    summation = 0.00
    for a in range(len(support_x)-1):
        for b in range(len(support_y)-1):
            ind_x = np.where((x>=support_x[a]) & (x<=support_x[a+1]))[0]
            ind_y = np.where((y>=support_y[b]) & (y<=support_y[b+1]))[0]

            px = len(ind_x) / len(x)
            py = len(ind_y) / len(x)
            pxy = len(np.where(np.in1d(ind_x, ind_y)==True)[0]) / len(x)

            if pxy>0.00: summation += pxy * np.log2(pxy / (px*py))


    return summation

In [8]:
def h_block(VA, BET, SUJETS, regress='r') :
    Full = e.Full_list(modes_bcp=[])
    modes_bcp = 'mean'
    tau = np.arange(1, 10001, 1)
    list_h = 1/tau

    r_va, r_bet = {}, {}
    for num, s in enumerate(SUJETS) :

        print(s, end=' ')
        
        if np.shape(BET[s])==(3, 200) :
            bet = []
            for b in range(3) : bet.extend(BET[s][b])
        else : bet = BET[s]
            
            
        if np.shape(VA[s])==(3, 200) :
            va = []
            for b in range(3) : va.extend(VA[s][b])
        else : va = VA[s]
        
        x = np.array(Full['bino'][Full.sujet==s].values.tolist())

        r_va[s], r_bet[s] = [], []
        for c in range(0, 3*200-150, 50) : r_va[s].append([]) ; r_bet[s].append([])

        for h in list_h :

            for d, c in enumerate(range(0, 3*200-150, 200)) :
                x_200 = x[c:c+200]
                p_hat_200 = np.zeros(200)

                liste = [0, 50, 100, 150, 200]
                for a in range(len(liste)-1) :
                    p_bar, r_bar, beliefs = bcp.inference(x_200[liste[a]:liste[a+1]], h=h, p0=.5, r0=1.)
                    p_hat_p, r_hat = bcp.readout(p_bar, r_bar, beliefs, mode=modes_bcp, p0=.5, fixed_window_size=40)
                    p_hat_200[liste[a]:liste[a+1]] = p_hat_p

                    
                    
                if regress=='r' :
                    r_va[s][d].append(r_(p_hat_200.tolist(), va[c:c+200]))
                    r_bet[s][d].append(r_(p_hat_200.tolist(), bet[c:c+200]))
                elif regress=='MI' :
                    r_va[s][d].append(mutual_information(p_hat_200.tolist(), va[c:c+200]))
                    r_bet[s][d].append(mutual_information(p_hat_200.tolist(), bet[c:c+200]))
                    
                
    recording = {}
    recording['list_h'] = list_h
    recording['list_tau'] = tau
    recording['r_va'] = r_va
    recording['r_bet'] = r_bet
    
    return recording

In [10]:
Full = e.Full_list(modes_bcp=[])

sujet = list(np.sort(list(set(Full.sujet))))

BET, VA = {}, {}
for s in sujet :
    BET[s] = Full['results'][Full.sujet==s].values.tolist()
    VA[s] = Full['va'][Full.sujet==s].values.tolist()
    
'''recording = h_block(VA, BET, sujet) 
file = os.path.join('parametre', 'list_h_block.pkl')
with open(file, 'wb') as fichier:
    f = pickle.Pickler(fichier)
    f.dump(recording)'''
    
recording = h_block(VA, BET, sujet, regress='MI') 

file = os.path.join('parametre', 'list_h_MI_block.pkl')
with open(file, 'wb') as fichier:
    f = pickle.Pickler(fichier)
    f.dump(recording)

AM BMC CS DC FM IP LB OP RS SR TN YK 

In [11]:
'''
data_scalling = e.Data_Scalling()
file = os.path.join('parametre', 'Data_Scaling.pkl')
with open(file, 'wb') as fichier:
    f = pickle.Pickler(fichier)
    f.dump(data_scalling)
'''

file = os.path.join('parametre', 'Data_Scaling.pkl')
with open(file, 'rb') as fichier:
    new_data = pickle.load(fichier, encoding='latin1')


Full = e.Full_list(modes_bcp=[])

sujet = list(np.sort(list(set(Full.sujet))))

'''
recording = h_block(new_data['new_va_sujet'], new_data['new_bet_sujet'], sujet) 

file = os.path.join('parametre', 'list_h_block_Scalling_sujet.pkl')
with open(file, 'wb') as fichier:
    f = pickle.Pickler(fichier)
    f.dump(recording)

recording = h_block(new_data['new_va_full'], new_data['new_bet_full'], sujet) 

file = os.path.join('parametre', 'list_h_block_Scalling_full.pkl')
with open(file, 'wb') as fichier:
    f = pickle.Pickler(fichier)
    f.dump(recording)'''

recording = h_block(new_data['new_va_sujet'], new_data['new_bet_sujet'], sujet, regress='MI') 

file = os.path.join('parametre', 'list_h_MI_block_Scalling_sujet.pkl')
with open(file, 'wb') as fichier:
    f = pickle.Pickler(fichier)
    f.dump(recording)

recording = h_block(new_data['new_va_full'], new_data['new_bet_full'], sujet, regress='MI') 

file = os.path.join('parametre', 'list_h_MI_block_Scalling_full.pkl')
with open(file, 'wb') as fichier:
    f = pickle.Pickler(fichier)
    f.dump(recording)

AM BMC CS DC FM IP LB OP RS SR TN YK AM BMC CS DC FM IP LB OP RS SR TN YK 